In [2]:
import os
import re
import sys
import sqlite3
import pandas as pd
sys.path.append('../src/')
from scraper.scraper import LiveTrailScraper
from results.results import Results
from database.create_db import Database
from database.database import Event, Race, Results


INFO: PACKAGE_DIR_PATH = /VSCode/personnel/MLTrail
INFO: DATA_DIR_PATH = /VSCode/personnel/MLTrail/data


In [3]:
import importlib
import results.results
importlib.reload(results.results)
from results.results import Results as RResults


In [4]:
import importlib
import database.database
importlib.reload(database.database)
from database.create_db import Database
from database.database import Event, Race, Results


In [5]:
import importlib
import scraper.scraper
importlib.reload(scraper.scraper)
from scraper.scraper import LiveTrailScraper
scraper = LiveTrailScraper()


In [6]:
db_path = os.path.join(os.getcwd(), '..', 'data', 'events.db')
data_path = os.path.join(os.getcwd(), '..', 'data', 'csv')


In [7]:
db: Database = Database.create_database(path=db_path)


INFO: /VSCode/personnel/MLTrail/examples/../data/events.db
INFO: Database already exists.


In [8]:
import warnings

# Suppress the XMLParsedAsHTMLWarning
warnings.filterwarnings("ignore", category=UserWarning, message=".*XMLParsedAsHTMLWarning.*")


In [102]:
from database import database as db_model
event = 'mbm'
year = '2023'
race = '42km'
event_id = db_model.Event.get_id_from_code_year(event, year)
print(event_id)
# MiM 2022: 456
# TGCC 2024: 616
# Marathon du Montblanc 2023: 394

394


In [99]:
db: Database = Database.create_database(path=db_path)
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    cursor.execute('''
        DROP TABLE IF EXISTS features
     ''')
    conn.commit()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS features (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                race_id TEXT,
                event_id INTEGER,
                bib TEXT,
                dist_total REAL,
                elevation_pos_total INTEGER,
                elevation_neg_total INTEGER,
                dist_segment REAL,
                dist_cumul REAL,
                elevation_pos_segment INTEGER,
                elevation_pos_cumul INTEGER,
                elevation_neg_segment INTEGER,
                elevation_neg_cumul INTEGER,
                time TEXT,
                FOREIGN KEY (race_id) REFERENCES races(race_id),
                FOREIGN KEY (event_id) REFERENCES events(event_id),
                FOREIGN KEY (bib) REFERENCES results(bib)
            )
     ''')
    conn.commit()
conn.close()

INFO: /VSCode/personnel/MLTrail/examples/../data/events.db
INFO: Database already exists.


In [100]:
db: Database = Database.create_database(path=db_path)
conn = sqlite3.connect(db.path, timeout=3600*10) # 10h timeout
try:
    with conn:
        cursor = conn.cursor()
        cursor.execute('''
            WITH ParsedTimes AS (
            SELECT
                tp.race_id,
                tp.event_id,
                tp.bib,
                cp.distance AS dist_cumul,
                cp.elevation_pos AS elevation_pos_cumul,
                cp.elevation_neg AS elevation_neg_cumul,
                tp.time,
                (SELECT MAX(distance) FROM control_points WHERE race_id = tp.race_id AND event_id = tp.event_id) AS dist_total,
                (SELECT MAX(elevation_pos) FROM control_points WHERE race_id = tp.race_id AND event_id = tp.event_id) AS elevation_pos_total,
                (SELECT MIN(elevation_neg) FROM control_points WHERE race_id = tp.race_id AND event_id = tp.event_id) AS elevation_neg_total,
                cp.elevation_pos - LAG(cp.elevation_pos, 1, 0) OVER (PARTITION BY tp.race_id, tp.event_id, tp.bib ORDER BY cp.control_point_id) AS elevation_pos_segment,
                cp.elevation_neg - LAG(cp.elevation_neg, 1, 0) OVER (PARTITION BY tp.race_id, tp.event_id, tp.bib ORDER BY cp.control_point_id) AS elevation_neg_segment,
                cp.distance - LAG(cp.distance, 1, 0) OVER (PARTITION BY tp.race_id, tp.event_id, tp.bib ORDER BY cp.control_point_id) AS dist_segment,
                -- only way to be able to work with >24h times
                (SUBSTR(tp.time, 1, 2) * 3600) + (SUBSTR(tp.time, 4, 2) * 60) + SUBSTR(tp.time, 7, 2) AS time_in_seconds,
                LAG((SUBSTR(tp.time, 1, 2) * 3600) + (SUBSTR(tp.time, 4, 2) * 60) + SUBSTR(tp.time, 7, 2), 1, 0) OVER (PARTITION BY tp.race_id, tp.event_id, tp.bib ORDER BY cp.control_point_id) AS prev_time_in_seconds
            FROM
                timing_points tp
            JOIN
                control_points cp ON tp.control_point_id = cp.control_point_id
            --WHERE
                --(tp.race_id = 'mim' AND
                --tp.event_id = 456) OR
                --(tp.race_id = 'classic' AND
                --tp.event_id = 616)
        ),
        TimeDifferences AS (
            SELECT
                race_id,
                event_id,
                bib,
                dist_cumul,
                elevation_pos_cumul,
                elevation_neg_cumul,
                dist_total,
                elevation_pos_total,
                elevation_neg_total,
                elevation_pos_segment,
                elevation_neg_segment,
                dist_segment,
                time,
                -- Compute time difference, adjust for times crossing 24 hours boundary
                printf('%02d:%02d:%02d', 
                    (time_in_seconds - prev_time_in_seconds ) / 3600,
                    ((time_in_seconds - prev_time_in_seconds ) % 3600) / 60,
                    ((time_in_seconds - prev_time_in_seconds ) % 3600) % 60
                ) AS time_segment
            FROM
                ParsedTimes
        ),
        -- Next part of the query is to add an extra line of full race data
        TotalStats AS (
            SELECT
                race_id,
                event_id,
                bib,
                dist_total AS dist_cumul,
                elevation_pos_total AS elevation_pos_cumul,
                elevation_neg_total AS elevation_neg_cumul,
                dist_total AS dist_total,
                elevation_pos_total AS elevation_pos_total,
                elevation_neg_total AS elevation_neg_total,
                dist_total AS dist_segment,
                elevation_pos_total AS elevation_pos_segment,
                elevation_neg_total AS elevation_neg_segment,
                (SELECT printf('%02d:%02d:%02d',
                    MAX(time_in_seconds) / 3600,
                    (MAX(time_in_seconds) % 3600) / 60,
                    (MAX(time_in_seconds) % 3600) % 60
                ) FROM ParsedTimes pt 
                WHERE pt.race_id = t.race_id AND pt.event_id = t.event_id AND pt.bib = t.bib) AS time_segment
            FROM
                TimeDifferences t
            WHERE
                dist_cumul = dist_total
        )
        INSERT INTO features (
            race_id,
            event_id,
            bib,
            dist_total,
            elevation_pos_total,
            elevation_neg_total,
            dist_segment,
            dist_cumul,
            elevation_pos_segment,
            elevation_pos_cumul,
            elevation_neg_segment,
            elevation_neg_cumul,
            time
        )
        SELECT
            race_id,
            event_id,
            bib,
            dist_total,
            elevation_pos_total,
            elevation_neg_total,
            dist_segment,
            dist_cumul,
            elevation_pos_segment,
            elevation_pos_cumul,
            elevation_neg_segment,
            elevation_neg_cumul,
            time_segment
        FROM
            TimeDifferences
        UNION ALL
        SELECT
            race_id,
            event_id,
            bib,
            dist_total,
            elevation_pos_total,
            elevation_neg_total,
            dist_segment,
            dist_cumul,
            elevation_pos_segment,
            elevation_pos_cumul,
            elevation_neg_segment,
            elevation_neg_cumul,
            time_segment
        FROM
            TotalStats
        ORDER BY
            race_id,
            event_id,
            bib,
            time_segment;

    ''')
        conn.commit()
finally:
    conn.close()
    

INFO: /VSCode/personnel/MLTrail/examples/../data/events.db
INFO: Database already exists.


## Work in progress

- [X] Fix over-24h races (multiple CPs at the same place to have resting times in ultras) (e.g. 616. transgrancanaria 2024 - 'classic')
- [ ] Remove all segments with distance below 0.5 km ? (multiple CPs at the same place to have resting times in ultras) (e.g. 646. trailnloue 2019 - '76km2j')
- [X] Check if it works for races with different departure times ? (this should use real times, not official) --> YES (e.g. 394. mbm 2023 - '42km')
- [ ] Only take into account if finish

In [109]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    df = pd.read_sql_query('''
        SELECT *
        FROM features
        WHERE
            bib = '101' AND
            event_id = 646 AND
            race_id = '76km2j'
        ORDER BY
            dist_cumul
    ''', conn)
conn.close()

df

,id,race_id,event_id,bib,dist_total,elevation_pos_total,elevation_neg_total,dist_segment,dist_cumul,elevation_pos_segment,elevation_pos_cumul,elevation_neg_segment,elevation_neg_cumul,time
0,3500866,76km2j,646,101,77.81,4926,-4817,7.12,7.12,281,281,-284,-284,00:37:26
1,3500868,76km2j,646,101,77.81,4926,-4817,5.87,12.99,446,727,-251,-535,00:41:40
2,3500872,76km2j,646,101,77.81,4926,-4817,10.89,23.88,580,1307,-661,-1196,01:25:51
3,3500873,76km2j,646,101,77.81,4926,-4817,7.89,31.77,714,2021,-519,-1715,01:42:53
4,3500871,76km2j,646,101,77.81,4926,-4817,6.39,38.16,419,2440,-354,-2069,01:13:32
5,3500864,76km2j,646,101,77.81,4926,-4817,0.00,45.71,0,2852,5,-2738,-5:-56:-40
6,3500875,76km2j,646,101,77.81,4926,-4817,7.55,45.71,412,2852,-674,-2743,11:15:31
7,3500874,76km2j,646,101,77.81,4926,-4817,3.99,49.70,433,3285,-46,-2784,06:27:48
8,3500869,76km2j,646,101,77.81,4926,-4817,8.41,58.11,511,3796,-784,-3568,00:43:22
9,3500870,76km2j,646,101,77.81,4926,-4817,6.24,64.35,531,4327,-323,-3891,00:54:57


In [101]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    df = pd.read_sql_query('''
        SELECT *
        FROM features
        WHERE
            bib = '603' AND
            event_id = 456 AND
            race_id = 'mim'
            --bib = '512' AND
            --event_id = 616 AND
            --race_id = 'classic'
        ORDER BY
            dist_cumul
    ''', conn)
conn.close()

df

,id,race_id,event_id,bib,dist_total,elevation_pos_total,elevation_neg_total,dist_segment,dist_cumul,elevation_pos_segment,elevation_pos_cumul,elevation_neg_segment,elevation_neg_cumul,time
0,8558085,mim,456,603,60.35,3325,-2120,8.47,8.47,357,357,-235,-235,00:58:30
1,8558090,mim,456,603,60.35,3325,-2120,14.60,23.07,693,1050,-468,-703,01:41:14
2,8558087,mim,456,603,60.35,3325,-2120,8.70,31.77,461,1511,-478,-1181,01:13:00
3,8558088,mim,456,603,60.35,3325,-2120,9.53,41.30,562,2073,-379,-1560,01:28:38
4,8558089,mim,456,603,60.35,3325,-2120,9.12,50.42,668,2741,-201,-1761,01:33:22
5,8558086,mim,456,603,60.35,3325,-2120,6.92,57.34,562,3303,-121,-1882,01:04:00
6,8558084,mim,456,603,60.35,3325,-2120,3.01,60.35,22,3325,-238,-2120,00:17:31
7,8558091,mim,456,603,60.35,3325,-2120,60.35,60.35,3325,3325,-2120,-2120,08:16:15


Example expected data (rounded) from ('penyagolosa', 2022)[456], 'mim', '603':

| **dist_total** | **elevation_pos_total** | **elevation_neg_total** | **dist_segment** | **dist_cumul** | **elevation_pos_segment** | **elevation_pos_cumul** | **elevation_neg_segment** | **elevation_neg_cumul** | **time_segment** |
|----------------|--------------------------|--------------------------|----------------|--------------------------|--------------------------|------------------|---------------------------|---------------------------|------------------|
| 60.4           | 3325                     | 2141                     | 8.5            | 8.5                      | 357                      | 357              | 228                       | 228                       | 0:58:00          |
| 60.4           | 3325                     | 2141                     | 14.6           | 23.1                     | 693                      | 1050             | 503                       | 731                       | 1:41:00          |
| 60.4           | 3325                     | 2141                     | 8.7            | 31.8                     | 461                      | 1511             | 475                       | 1206                      | 1:13:00          |
| 60.4           | 3325                     | 2141                     | 9.5            | 41.3                     | 562                      | 2073             | 387                       | 1593                      | 1:29:00          |
| 60.4           | 3325                     | 2141                     | 9.1            | 50.4                     | 668                      | 2741             | 176                       | 1769                      | 1:33:00          |
| 60.4           | 3325                     | 2141                     | 6.9            | 57.3                     | 562                      | 3303             | 125                       | 1894                      | 1:04:00          |
| 60.4           | 3325                     | 2141                     | 3.1            | 60.4                     | 22                       | 3325             | 247                       | 2141                      | 0:18:00          |
| 60.4           | 3325                     | 2141                     | 60.4           | 60.4                     | 3325                     | 3325             | 2141                      | 2141                      | 8:16:16          |


____

In [104]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    df = pd.read_sql_query('''
        SELECT *
        FROM features
        WHERE
            --bib = '603' AND
            --event_id = 456 AND
            --race_id = 'mim'
            bib = '908' AND
            event_id = 394 AND
            race_id = '42km'
            --bib = '512' AND
            --event_id = 616 AND
            --race_id = 'classic'
        ORDER BY
            dist_cumul
    ''', conn)
conn.close()

df

,id,race_id,event_id,bib,dist_total,elevation_pos_total,elevation_neg_total,dist_segment,dist_cumul,elevation_pos_segment,elevation_pos_cumul,elevation_neg_segment,elevation_neg_cumul,time
0,2259507,42km,394,908,45.56,2755,-2755,0.00,0.00,0,0,0,0,00:00:00
1,2259509,42km,394,908,45.56,2755,-2755,9.68,9.68,588,588,-364,-364,01:00:30
2,2259513,42km,394,908,45.56,2755,-2755,10.28,19.96,1055,1643,-309,-673,01:51:32
3,2259508,42km,394,908,45.56,2755,-2755,4.87,24.83,8,1651,-741,-1414,00:41:44
4,2259512,42km,394,908,45.56,2755,-2755,8.19,33.02,564,2215,-376,-1790,01:49:28
5,2259510,42km,394,908,45.56,2755,-2755,3.86,36.88,469,2684,-35,-1825,01:17:58
6,2259511,42km,394,908,45.56,2755,-2755,8.68,45.56,71,2755,-930,-2755,01:27:31
7,2259514,42km,394,908,45.56,2755,-2755,45.56,45.56,2755,2755,-2755,-2755,08:08:43


In [63]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    df = pd.read_sql_query('''
        SELECT *
        FROM results
        WHERE
            bib = '512' AND
            event_id = 616 AND
            race_id = 'classic'
    ''', conn)
conn.close()

df

,race_id,event_id,position,cat_position,full_cat_position,bib,surname,name,sex_category,full_category,time
0,classic,616,325,290,18,512,NACHER CASTELLET,Victor,Male,EL H,25:59:58


In [64]:
conn = sqlite3.connect(db.path)
with conn:
    cursor = conn.cursor()
    df = pd.read_sql_query('''
        SELECT *
        FROM timing_points
        WHERE
            bib = '512' AND
            event_id = 616 AND
            race_id = 'classic'
    ''', conn)
conn.close()

df

,timing_point_id,control_point_id,race_id,event_id,bib,time
0,21074850,5752,classic,616,512,01:19:14
1,21074851,5753,classic,616,512,02:31:01
2,21074852,5754,classic,616,512,04:31:51
3,21074853,5755,classic,616,512,06:54:41
4,21074854,5756,classic,616,512,09:17:41
5,21074855,5757,classic,616,512,12:05:50
6,21074856,5758,classic,616,512,14:51:32
7,21074857,5759,classic,616,512,17:29:44
8,21074858,5760,classic,616,512,18:12:44
9,21074859,5761,classic,616,512,20:57:21
